#### Lesson Objective:
* learn how to read and play sound files in a Jupyter notebook
* learn how to create a spectrogram from a signal
* learn how to compare two signals

#### Reading and Playing Sound

In [ ]:
# import common packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

For this tutorial we will use a [dataset](https://zenodo.org/record/574438#.WVyQatPyvzI) extracted from the [xeno-canto](http://www.xeno-canto.org/) database of bird recordings.

The recordings are in `.wav` format. We can use the `io` submodule of `scipy` to read them.

In [ ]:
from scipy.io import wavfile

In [ ]:
birdsound = wavfile.read('data/PicidaeDataset/DendrocoposMedius-song/XC26678-DendrocoposMedius-song-1.wav')

The output object is a tuple of a frequency and a 1D signal (it is possible that the recording contains more than one channels, and then we have a 2D array)

In [ ]:
# structure of the sound object
str(birdsound)

Plotting the signal: 

to obtain the time axis in seconds we need to divide the frame number by the frequency.

In [ ]:
plt.plot(np.linspace(0,len(birdsound[1])/birdsound[0], len(birdsound[1])),birdsound[1])
plt.xlabel('Time in s')

We can play the sound directly in the Jupyter notebook:

In [ ]:
from IPython.display import Audio

In [ ]:
Audio('data/PicidaeDataset/DendrocoposMedius-song/XC26678-DendrocoposMedius-song-1.wav')

#### Spectrogram

It is useful to look at the spectrogram of the signal as it shows details at different frequencies.

This can be achieved through the `specgram` function in `matplotlib`.

In [ ]:
Pxx, freqs, bins, im = plt.specgram(birdsound[1],Fs = birdsound[0])
plt.title('Song Spectrogram')

In general there are different approaches working with sound:

* working with the signal directly
* working with the spectrogram
* working with features extracted from the singal/spectrogram

In order to automate the process of reading files, we can use the `glob` function, which gives us all filenames in a given directory.

In [ ]:
from glob import glob

In [ ]:
filenames = glob('data/PicidaeDataset/DendrocoposMinor-drumming/*')

In [ ]:
filenames

The filenames may not be ordered, but we can order them for consistency:

In [ ]:
filenames = sorted(filenames)
filenames

In [ ]:
songs = [wavfile.read(file) for file in filenames]

In [ ]:
plt.figure(figsize = [10,5])
plt.plot(songs[0][1])
plt.plot(songs[1][1],'r')

To compare two different drummings we need to first align them. We can achieve that by moving one of the signals to be maximally correlated with the other one.

We can efficiently calculate cross-correlation using Fast Fourier Transforms:

In [ ]:
from scipy import signal

In [ ]:
cc = signal.fftconvolve(np.abs(songs[0][1]),np.abs(songs[1][1][::-1]), mode='full')
l  = len(songs[1][1])

# finding the offset of maximum correlation
offset = cc.argmax() + l  - len(cc)
offset

In [ ]:
# shift signal 1 by the ofset to align with signal 2
plt.plot(songs[0][1][-offset:])
plt.plot(songs[1][1],'r')

Exercise: 

* define a similarity measure between two bird sounds
* calculate the similarity for every pair of two sounds
* build a network of bird sounds and explore its connectivity
* mix sounds from different birds: does your similarity separate the different species?